# Parsing docx, doc in pandas 

In [29]:
from docx import Document
import pandas as pd

def docx_to_dataframe(file_path):
    doc = Document(file_path)
    data = []
    
    for table in doc.tables:
        for row in table.rows:
            row_data = [cell.text.strip() for cell in row.cells]
            data.append(row_data)
    
    return pd.DataFrame(data)

df = docx_to_dataframe('./files/5.docx')

In [31]:
df

,0,1,2,3,4
0,№\nп/п,Наименование,Технические характеристики,Ед. изм.,Кол-во
1,,Конвектор,Максимальная мощность:2000 Вт \nРегулировка те...,шт,22
2,,Конвектор,Максимальная мощность:1500 Вт \nУправление:эле...,шт,15
3,,Светильник светодиодный \nвстраиваемый в потол...,Мощность не менее 36 Вт\nСтепень пылевлагозащи...,шт,53
4,,Светильник светодиодный уличный сд ДКУ-03 или...,"Мощность не менее 120 Вт\nСветовой поток, лм: ...",шт,111
5,,"Светодиодный уличный прожектор Boxbot, 50 Вт 6...",Мощность не менее 50 Вт\nПлощадь освещения не ...,шт,20
6,,Светодиодный светильник СПО или эквивалент,Вид крепления накладной\nТип настенные/потолоч...,шт,60
7,,Лампа светодиодная,Тип цоколя: E27 \nМощность: не менее 9 Вт\nТе...,шт,250
8,,"Кабель ВВГ-3х2,5",ГОСТ 31996-2012.,м,52
9,,Распределительная коробка карболитовая 4-х рож...,Длина: 95±2 мм.\nШирина: 95±2 мм.\nГлубина: 40...,шт.,5


# Xyita ne rabotaet, libo nado nastroit'

In [1]:
import pymupdf
doc = pymupdf.open("./files/4.pdf")

In [ ]:
for page in doc[:1]:
    table_finder = page.find_tables()

In [33]:
table_finder.tables[1].to_pandas()

,Ламп,а светодиодная,Col2,Col3
0,ЭРА,ECO,LED,T8-
1,None,W-865-G13,None,None


# Find attributes from pdf with llm and markdown

In [39]:
import pymupdf4llm
md_text = pymupdf4llm.to_markdown("./files/4.pdf")

In [40]:
md_text

'РАЗДЕЛ 1. ПЕРЕЧЕНЬ ТОВАРОВ И ОБЩИХ ТРЕБОВАНИЙ\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n|№<br>п/п|Наименование-1|Основные технические<br>характеристики товара-|Ед.<br>изм.|Кол-<br>во-5|Срок<br>поставки-6|Место<br>поставки|Объем гарантий и<br>гарантийный срок-<br>7|Код ОКПД-2|\n|---|---|---|---|---|---|---|---|---|\n|1.<br>|Лампа СКЛ 14-Б-Л-<br>2-110 ТУ<br>ЕНСК.433137.14 ТУ<br>(или аналог) <br>GID 1990187 <br>SAP ERP 1051121189|Рабочее напряжение не менее<br>109В не более 110В.; <br>Цвет свечения зеленый; <br>Установочный диаметр не<br>менее d22 <br>Сила света  не менее 36 мКд; <br>Рабочий ток не менее 9,9А не<br>более 10мА; <br>Степень защиты не менее IP54; <br>Режим свечения: постоянное|шт|12|30.10.2025<br>с правом<br>досрочной<br>поставки|Чукотский<br>АО, г.<br>Певек,<br>ПАТЭС,<br>улица<br>Энергетико<br>в, стр. 6|Гарантийный срок<br>не менее 24<br>месяцев с даты<br>приёмки<br>продукции на<br>складе покупателя|27.40.39.190 <br>Арматура<br>осветительная<br>прочая, не<br>включен

In [48]:
len(md_text)

14371

In [41]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-5-mini")

In [52]:
prompt = f"""Анализируй предоставленную таблицу в формате Markdown. Твоя задача — точно извлечь значение из указанного поля.

**Правила обработки:**
1.  Внимательно сопоставь заголовки столбцов таблицы с запрошенным полем.
2.  Игнорируй разрывы строк внутри ячеек (символы `<br>` или переносы). Считай содержимое ячейки единым текстом.
3.  Если запрошенное поле отсутствует в таблице или значение не указано, верни "Данные отсутствуют".

**Таблица для анализа:**
{md_text[:]}
**Задание:**
Извлеки значение из столбца **"Наименование-1"** для всех товаров.

**Формат ответа:**
Верни ТОЛЬКО извлеченные значения в виде строки, разделенные ';'."""

In [53]:
res = llm.invoke(prompt)

In [55]:
res.content.split(";")

['Лампа СКЛ 14-Б-Л-2-110 ТУ ЕНСК.433137.14 ТУ (или аналог) GID 1990187 SAP ERP 1051121189',
 ' Трансформатор ТЭ-105 SQ0360-0002 (или аналог) GID 2432172 SAP ERP 1010001041',
 ' Лампа светодиодная 1038920 PLED-HP-T120 40Вт E27 4000K JazzWay (или аналог) GID 2134184',
 ' Лампа светодиодная ECO LED T8-10W-840-G13-600mm Б0032974 ЭРА (или аналог) GID 1811011 SAP ERP 1011079938',
 ' Прожектор судовый DH/1 светодиодная матрица 350мм 150Вт дальность 2200м Циклон (или аналог) GID 3187303 SAP ERP 1011356761',
 ' Лампа сетодиодная LED-A60-standard 15Вт 230В Е27 4000К 1350Лм 4690612002101 ASD (или аналог) GID 1509936 SAP ERP 1011008392',
 ' Лампа светодиодная ЭРА ECO LED T8-18W-865-G13-1200mm Б0032977 ЭРА (или аналог) GID 1809748 SAP ERP 1011079937',
 ' Светильник FREGAT LED 110 (W) 5000К 1426000020 107Вт ТУ 3461-022-88466159-14 Световые Технологи (или аналог) GID 1035273 SAP ERP 710976172',
 ' Светильник светодиодный Titan 270 CW T IP65 ВСЗ Луч (или аналог) GID 1348210 SAP ERP 710976194',
 ' Свет